# 3 Extract facial data from videos.

We use a range of libraries to extract facial data from the videos. The main library is [DeepFace](https://github.com/serengil/deepface) but we also considered FER - [Facial Expression Recognition](https://github.com/justinshenk/fer).

`pip install deepface`



## 3.1 DeepFace



In [ ]:
from deepface import DeepFace
import pandas as pd
import pprint
import cv2
import os
import time
import utils


## Test data

In [ ]:
videos_in = os.path.join("..","LookitLaughter.test")
demo_data = os.path.join("..","data", "demo")
data_out = os.path.join("..","data","1_interim")


test_video = "2UWdXP.joke1.rep2.take1.Peekaboo.mp4"
test_video = "6c6MZQ.joke1.rep1.take1.ThatsNotAHat.mp4"

test_image = "peekaboo.png"
test_image2 = "mother-and-baby.jpg"

## Full data

In [ ]:
videos_in = os.path.join("..","..","LookitLaughter.full.videos")
temp_out = os.path.join("..","..","LookitLaughter.full.data","0_temp")
data_out = os.path.join("..","..","LookitLaughter.full.data","1_interim")
videos_out = os.path.join("..","..","LookitLaughter.full.data","2_final")

In [ ]:
#just playing around to try out different models built into deepface
backends = [
  'opencv', 
  'ssd', 
  'dlib', 
  'mtcnn', 
  'retinaface', 
  'mediapipe',
  'yolov8',
  'yunet',
]
imagepath = os.path.join(demo_data,test_image2)
result = DeepFace.analyze(img_path=imagepath, enforce_detection = False, detector_backend = backends[1])

for r in result:
    pprint.pprint(r)


In [ ]:
import matplotlib.pyplot as plt

imagepath = os.path.join(demo_data,test_image2)
#show image
image1 = cv2.imread(imagepath)
#plt.imshow(image1)
#plt.show()



In [ ]:
plt.imshow(image1)
#plt.show()


In [ ]:
def getfacedataforvideo(video_path, backend = 'ssd'):
    print("Processing video: ", video_path)
    cap = cv2.VideoCapture(video_path)
    success, image = cap.read()
    if not success:
        print("Failed to read video: ", video_path)
        return None
    facesdf = utils.createfacesdf()
    frameidx = 0
    fails = []
    while success:
        try:  
            faces = DeepFace.analyze(img_path = image, 
                                    enforce_detection = True, 
                                    actions = ('age','gender','emotion'),
                                    detector_backend = backend)
        except:
            #print(f"deepface.analyse failed for frame {frame}" )
            fails.append(frameidx)
            faces = []
        if len(faces) > 0:
            facesdf = utils.addfacestodf(facesdf,frameidx,faces)
        success,image = cap.read()
        frameidx += 1
    
    print(f"Failed to process {len(fails)}/{frameidx} ({(round(100*len(fails)/frameidx, 1))}%) frames")
    cap.release()
    return facesdf

In [ ]:
processedvideos = utils.getProcessedVideos(data_out)
processedvideos.head()

In [ ]:
forceFaces = False
backend = "ssd"

for index, r in processedvideos.iterrows():
    if forceFaces or pd.isnull(r["Faces.file"]) or not os.path.exists(r["Faces.file"]):
        filepath = os.path.join(videos_in, r["VideoID"])
        facesdf = getfacedataforvideo(filepath, backend = backend)
        if not facesdf is None:
            stemname = os.path.splitext(r["VideoID"])[0]
            facefile =  os.path.join(data_out, stemname + f".faces.{backend}.csv")
            facesdf.to_csv(facefile, index=False)
            r["Faces.file"] = facefile
            r["Faces.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        else:
            r["Faces.file"] = ""
            r["Faces.when"] = ""        
        #update this row in processedvideos dataframe
        processedvideos.loc[index] = r
    else:
        print(f"Already processed {r['VideoID']}")
utils.saveProcessedVideos(processedvideos, data_out)

In [ ]:
processedvideos.head()

#TODO 0001 - add code to match the person labels for the faces to person labels from pose detection (step 1)
# https://github.com/InfantLab/babyjokes/issues/1

## create a normed version of the csv file

similar to step 1.


In [ ]:
import calcs, utils
facecolsx, facecolsy = utils.getfacecols()

#Normalise all the x,y coordinates

#loop through each row of processedvideos and create a new dataframe & csv file with normalised coordinates
for index, row in processedvideos.iterrows():
    keypointsdf = pd.read_csv(row["Faces.file"])
    normedkeypointsdf = calcs.normaliseCoordinates(keypointsdf, facecolsx, facecolsy, row["Height"],row["Width"])
    #save keypointsdf as csv
    stemname = os.path.splitext(row["Faces.file"])[0]
    normedkeypointspath = os.path.join(stemname + "_normed.csv")
    processedvideos.at[index,"Faces.normed"] = normedkeypointspath
    normedkeypointsdf.to_csv(normedkeypointspath, index=False)  


utils.saveProcessedVideos(processedvideos, data_out)
